# Olympics

In this example, we will be exploring the weights and heights of olympic athletes by using hvplot and pandas.

In [1]:
import hvplot.pandas  # noqa\n"
import pandas as pd
    

## Data loading

First, we will load in the data.

In [2]:
df = pd.read_csv('https://media.githubusercontent.com/media/srungta/120-years-of-olympics/master/data/athlete_events.csv')
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


## Analysis

Since we are focusing on the weight and height columns of our dataset, let's remove any NaN values that are in those columns.

In [3]:
df = df.dropna(subset=["Weight", "Height"])

One of the first things to look at is the the average height of each sport. 

In [4]:
avg_height_by_sport = df.groupby('Sport')['Height'].mean()
avg_height_by_sport.sort_values(inplace=True)
avg_height_by_sport.head()

Sport
Gymnastics             162.875641
Trampolining           166.563758
Diving                 166.662196
Rhythmic Gymnastics    167.795122
Weightlifting          167.822726
Name: Height, dtype: float64

Using hvplot, we can create a horizontal bar chart by using `.hvplot.barh`. Then we add labels to our chart and select the basketball and gymnastics as they have the highest and lowest heights. We finally combine everything by overlaying the chart, labels, and select together to create our chart through the use of `*` operator.

In [5]:
bar = avg_height_by_sport.hvplot.barh(by='Sport', stacked=True, height=600)
txt = avg_height_by_sport.hvplot.labels("Sport", "Height", "{Height:.0f} cm", invert=True, text_align="left", text_baseline="middle", text_color="darkred")
bar_sel = bar.select(Sport=["Basketball", "Gymnastics"]).opts(fill_color="darkred")
(bar * bar_sel * txt.select(Sport=["Basketball", "Gymnastics"])).opts(title="Average Height of each Olympic Sport, a 28 cm difference")

:Overlay
   .Bars.Height.I  :Bars   [Sport]   (Height)
   .Bars.Height.II :Bars   [Sport]   (Height)
   .Labels.I       :Labels   [Sport,Height]   (label)

We can also create a line plot between difference the average heights of basketball and gymanstic olympians for each year. It's interesting to see that the difference has been growing wider with it peaking at 1992. 

In [6]:
avg_basketball_height = df[df["Sport"] == "Basketball"].groupby("Year")["Height"].mean()
avg_gymnastics_height = df[df["Sport"] == "Gymnastics"].groupby("Year")["Height"].mean()
avg_height_diff = avg_basketball_height - avg_gymnastics_height
avg_height_diff.hvplot.line()


:Curve   [Year]   (Height)

We can also take a look at the height and weights at the same time for each medal and sport combination. We were able to overlay the plot earlier using `*` but we can also arrange the two plots together using `+`. To make them vertical we use `.cols(1)`.


In [10]:
df = df.sort_values("Year")
df2 = df.dropna(subset=["Medal"])
height_plot = df2.hvplot("Year", "Height", groupby=["Medal", "Sport"])
weight_plot = df2.hvplot("Year", "Weight", groupby=["Medal", "Sport"])
combined_plots = height_plot + weight_plot
combined_plots.cols(1)

:Layout
   .DynamicMap.I  :DynamicMap   [Medal,Sport]
      :Curve   [Year]   (Height)
   .DynamicMap.II :DynamicMap   [Medal,Sport]
      :Curve   [Year]   (Weight)

We can further narrow down the data by looking at specific events instead of sports. 

In [11]:
df2.hvplot("Year", "Weight", groupby=["Medal", "Event"])

:DynamicMap   [Medal,Event]
   :Curve   [Year]   (Weight)

We can even take a step further by looking at the difference in weight and height between medalists and non-medalists. First, we will separate the data by having the averages of medalists and non-medalists. We use the `*` operator to overlay the medalists and non-medalists data for both the height and weight plots. Then we use the `+` operator to arrange the two plots. 

This gives us very interesting graphs as we can see there is a distinct difference of height and weight between medalists and non-medalists of the athletics men's 100 metres event.

In [12]:
medalist_avg = df[df["Medal"].notna()].groupby(["Year", "Event"]).agg({"Height": "mean", "Weight": "mean"})
not_medalist_avg = df[df["Medal"].isna()].groupby(["Year", "Event"]).agg({"Height": "mean", "Weight": "mean"})

height_medalist_plot = medalist_avg.hvplot.line(x="Year", y="Height", groupby=["Event"], label='Medalist')
height_not_medalist_plot = not_medalist_avg.hvplot.line(x="Year", y="Height", groupby=["Event"], color="red", label='Non-Medalist')
height_plot = (height_medalist_plot * height_not_medalist_plot).opts(title="Height", legend_position="bottom_right")

weight_medalist_plot = medalist_avg.hvplot.line(x="Year", y="Weight", groupby=["Event"], label='Medalist')
weight_not_medalist_plot = not_medalist_avg.hvplot.line(x="Year", y="Weight", groupby=["Event"], color="red", label='Non-Medalist')
weight_plot = (weight_medalist_plot * weight_not_medalist_plot).opts(title="Weight", legend_position="bottom_right")
weight_plot

combined_plots = height_plot + weight_plot

combined_plots.cols(1)

:Layout
   .DynamicMap.I  :DynamicMap   [Event]
      :Overlay
         .Curve.Medalist                  :Curve   [Year]   (Height)
         .Curve.Non_hyphen_minus_Medalist :Curve   [Year]   (Height)
   .DynamicMap.II :DynamicMap   [Event]
      :Overlay
         .Curve.Medalist                  :Curve   [Year]   (Weight)
         .Curve.Non_hyphen_minus_Medalist :Curve   [Year]   (Weight)